In [ ]:
!pip install caer canaro

In [3]:
import os
import caer
import canaro
import numpy as np
import cv2 as cv
import gc

In [3]:
IMG_SIZE=(80,80)
channels = 1
char_path= r'../input/the-simpsons-characters-dataset/simpsons_dataset'


In [5]:
char_dict={}
for char in os.listdir(char_path):
    char_dict[char]=len(os.listdir(os.path.join(char_path,char)))

#sort this dict in desc order
char_dict = caer.sort_dict(char_dict,descending=True)
char_dict

In [6]:
characters=[]
count=0
for i in char_dict:
    characters.append(i[0])
    count+=1
    if count>=10:
        break
characters

In [7]:
train = caer.preprocess_from_dir(char_path,characters,channels=channels,IMG_SIZE=IMG_SIZE,isShuffle=True)

In [8]:
len(train)

In [11]:
import matplotlib.pyplot as plt
plt.figure(figsize=(30,30))
plt.imshow(train[0][0],cmap='gray')
plt.show()


In [13]:
featureSet,labels= caer.sep_train(train,IMG_SIZE=IMG_SIZE)

In [14]:
from tensorflow.keras.utils import to_categorical
#normalize the featureSet
featureSet = caer.normalize(featureSet)
labels = to_categorical(labels,len(characters))


In [15]:
x_train,x_val,y_train,y_val= caer.train_val_split(featureSet,labels,val_ratio=0.2)

In [18]:
del train
del featureSet
del labels
gc.collect()


In [19]:
BATCH_SIZE=32
EPOCHS= 10

In [22]:
datagen = canaro.generators.imageDataGenerator()
train_gen = datagen.flow(x_train,y_train,batch_size= BATCH_SIZE)

In [28]:
model = canaro.models.createSimpsonsModel(IMG_SIZE=IMG_SIZE,channels= channels,output_dim=len(characters),
                                          loss='binary_crossentropy',decay=1e-6,learning_rate=0.001,
                                          momentum=0.9,nesterov=True)

In [30]:
model.summary()

In [32]:
from tensorflow.keras.callbacks import LearningRateScheduler
callbacks_list= [LearningRateScheduler(canaro.lr_schedule)]


In [2]:
training = model.fit(train_gen, steps_per_epoch=len(x_train)//BATCH_SIZE,
                     epochs= EPOCHS,
                    validation_data=(x_val,y_val),
                    validation_steps= len(y_val)//BATCH_SIZE,
                    callbacks=callbacks_list)